In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.metrics import *

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [8]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 175ms/step - loss: 0.6920 - accuracy: 0.5389 - precision: 0.5086 - recall: 0.4813 - auc: 0.5763
Epoch 2/30
58/58 [==============================] - 10s 177ms/step - loss: 0.6118 - accuracy: 0.6550 - precision: 0.6628 - recall: 0.5374 - auc: 0.7241
Epoch 3/30
58/58 [==============================] - 10s 178ms/step - loss: 0.5210 - accuracy: 0.7448 - precision: 0.7305 - recall: 0.7220 - auc: 0.8228
Epoch 4/30
58/58 [==============================] - 10s 177ms/step - loss: 0.4552 - accuracy: 0.7744 - precision: 0.7681 - recall: 0.7430 - auc: 0.8678
Epoch 5/30
58/58 [==============================] - 10s 178ms/step - loss: 0.3642 - accuracy: 0.8434 - precision: 0.8246 - recall: 0.8458 - auc: 0.9197
Epoch 6/30
58/58 [==============================] - 10s 176ms/step - loss: 0.3048 - accuracy: 0.8675 - precision: 0.8545 - recall: 0.8645 - auc: 0.9438
Epoch 7/30
58/58 [==============================] - 10s 178ms/step - loss: 0.2485 - accu

58/58 [==============================] - 11s 183ms/step - loss: 0.0335 - accuracy: 0.9901 - precision_1: 0.9861 - recall_1: 0.9930 - auc_1: 0.9985
Epoch 24/30
58/58 [==============================] - 11s 184ms/step - loss: 0.0253 - accuracy: 0.9967 - precision_1: 0.9953 - recall_1: 0.9977 - auc_1: 0.9987
Epoch 25/30
58/58 [==============================] - 11s 183ms/step - loss: 0.0396 - accuracy: 0.9912 - precision_1: 0.9953 - recall_1: 0.9860 - auc_1: 0.9985
Epoch 26/30
58/58 [==============================] - 11s 182ms/step - loss: 0.0221 - accuracy: 0.9912 - precision_1: 0.9861 - recall_1: 0.9953 - auc_1: 0.9998
Epoch 27/30
58/58 [==============================] - 11s 183ms/step - loss: 0.0373 - accuracy: 0.9880 - precision_1: 0.9906 - recall_1: 0.9836 - auc_1: 0.9982
Epoch 28/30
58/58 [==============================] - 10s 180ms/step - loss: 0.0617 - accuracy: 0.9781 - precision_1: 0.9766 - recall_1: 0.9766 - auc_1: 0.9961
Epoch 29/30
58/58 [==============================] - 11s 1

58/58 [==============================] - 11s 182ms/step - loss: 0.0575 - accuracy: 0.9814 - precision_3: 0.9746 - recall_3: 0.9860 - auc_3: 0.9984
Epoch 15/30
58/58 [==============================] - 11s 184ms/step - loss: 0.0511 - accuracy: 0.9836 - precision_3: 0.9882 - recall_3: 0.9766 - auc_3: 0.9982
Epoch 16/30
58/58 [==============================] - 11s 183ms/step - loss: 0.0799 - accuracy: 0.9759 - precision_3: 0.9743 - recall_3: 0.9743 - auc_3: 0.9952
Epoch 17/30
58/58 [==============================] - 11s 182ms/step - loss: 0.0833 - accuracy: 0.9869 - precision_3: 0.9860 - recall_3: 0.9860 - auc_3: 0.9938
Epoch 18/30
58/58 [==============================] - 10s 181ms/step - loss: 0.2262 - accuracy: 0.9332 - precision_3: 0.9218 - recall_3: 0.9369 - auc_3: 0.9760
Epoch 19/30
58/58 [==============================] - 11s 184ms/step - loss: 0.2688 - accuracy: 0.9080 - precision_3: 0.8891 - recall_3: 0.9182 - auc_3: 0.9607
Epoch 20/30
58/58 [==============================] - 11s 1

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 19s 320ms/step - loss: 0.6560 - accuracy: 0.6046 - precision_5: 0.5777 - recall_5: 0.5818 - auc_5: 0.6486
Epoch 2/30
58/58 [==============================] - 19s 324ms/step - loss: 0.4764 - accuracy: 0.7831 - precision_5: 0.7889 - recall_5: 0.7336 - auc_5: 0.8603
Epoch 3/30
58/58 [==============================] - 19s 323ms/step - loss: 0.3425 - accuracy: 0.8510 - precision_5: 0.8288 - recall_5: 0.8598 - auc_5: 0.9270
Epoch 4/30
58/58 [==============================] - 19s 323ms/step - loss: 0.3078 - accuracy: 0.8784 - precision_5: 0.8644 - recall_5: 0.8785 - auc_5: 0.9408
Epoch 5/30
58/58 [==============================] - 19s 326ms/step - loss: 0.1999 - accuracy: 0.9299 - precision_5: 0.9136 - recall_5: 0.9393 - auc_5: 0.9728
Epoch 6/30
58/58 [==============================] - 19s 322ms/step - loss: 0.1900 - accuracy: 0.9398 - precision_5: 0.9307 - recall_5: 0.9416 - auc_5: 0.9742
Epoch 7/30
58/58 [==============================] - 

58/58 [==============================] - 19s 321ms/step - loss: 0.0527 - accuracy: 0.9803 - precision_6: 0.9858 - recall_6: 0.9720 - auc_6: 0.9962
Epoch 23/30
58/58 [==============================] - 19s 323ms/step - loss: 0.0250 - accuracy: 0.9912 - precision_6: 0.9930 - recall_6: 0.9883 - auc_6: 0.9997
Epoch 24/30
58/58 [==============================] - 19s 322ms/step - loss: 0.1754 - accuracy: 0.9387 - precision_6: 0.9189 - recall_6: 0.9533 - auc_6: 0.9821
Epoch 25/30
58/58 [==============================] - 19s 321ms/step - loss: 0.0298 - accuracy: 0.9912 - precision_6: 0.9884 - recall_6: 0.9930 - auc_6: 0.9997
Epoch 26/30
58/58 [==============================] - 19s 320ms/step - loss: 0.0063 - accuracy: 1.0000 - precision_6: 1.0000 - recall_6: 1.0000 - auc_6: 1.0000
Epoch 27/30
58/58 [==============================] - 19s 319ms/step - loss: 0.0037 - accuracy: 1.0000 - precision_6: 1.0000 - recall_6: 1.0000 - auc_6: 1.0000
Epoch 28/30
58/58 [==============================] - 19s 3

58/58 [==============================] - 19s 322ms/step - loss: 0.0207 - accuracy: 0.9978 - precision_8: 0.9977 - recall_8: 0.9977 - auc_8: 0.9995
Epoch 14/30
58/58 [==============================] - 19s 322ms/step - loss: 0.0121 - accuracy: 0.9978 - precision_8: 0.9977 - recall_8: 0.9977 - auc_8: 1.0000
Epoch 15/30
58/58 [==============================] - 19s 323ms/step - loss: 0.0075 - accuracy: 0.9989 - precision_8: 0.9977 - recall_8: 1.0000 - auc_8: 1.0000
Epoch 16/30
58/58 [==============================] - 18s 318ms/step - loss: 0.0073 - accuracy: 0.9978 - precision_8: 0.9977 - recall_8: 0.9977 - auc_8: 0.9999
Epoch 17/30
58/58 [==============================] - 19s 319ms/step - loss: 0.0051 - accuracy: 0.9978 - precision_8: 0.9977 - recall_8: 0.9977 - auc_8: 1.0000
Epoch 18/30
58/58 [==============================] - 19s 320ms/step - loss: 0.0044 - accuracy: 0.9989 - precision_8: 0.9977 - recall_8: 1.0000 - auc_8: 1.0000
Epoch 19/30
58/58 [==============================] - 19s 3